# Преобразование csv Sound Speed Manager в .svp PDS2000

Скрипт для конвертации обработанных в SoundSpeedManager .TXT данных Valeport MiniSVP в формат .svp для проекта PDS2000

Для данного скрипта необходим журнал SVP зондирований, с шапкой: ['NumO', 'Num', 'S/N', 'FileName', 'Date', 'TimeUTC', 'DateTimeUTCOpCPN', 'DateTimeUTCProbe', 'Station', 'Location', 'latN', 'lonE']

S/N - серийный номер зонда; FileName - Имя файла .TXT; DateTimeUTCOpCPN -  время станции зондирования в UTC (не время из шапки файла .TXT; Station - название/номер станции

In [8]:
import os
from cli import cli

def read_my_svp_logbook(path):
    sn_list = []
    fn_list = []
    dt_list = []
    st_list = []

    with open(path) as f:
        file_content = f.read().splitlines()
        header = 1
        for line in file_content:
            if header == 1:
                pass
            else:
                line_content = line.split(',')
                # header: ['NumO', 'Num', 'S/N', 'FileName', 'Date', 'TimeUTC', 'DateTimeUTCOpCPN', 'DateTimeUTCProbe', 'Station', 'Location', 'latN', 'lonE']
                # header: Num	S/NorType	FileName	DatetimeUTC	Station	X	Y	Lat	Lon
                sn_list.append(line_content[1])
                fn_list.append(line_content[2])
                dt_list.append(line_content[3])
                st_list.append(line_content[4])
            header = 0
        return sn_list, fn_list, dt_list, st_list

def modify_output_file_name(sn_or_type, logpath, output_path, filetype='SSMminiSVP'):
    # Read logbook
    sn_list, fn_list, dt_list, st_list = read_my_svp_logbook(logpath)
    
    # Get the file name
    out_folder_path, out_fname_ext2_ext1 = os.path.split(output_path)
    out_fname_ext2, out_ext1 = os.path.splitext(out_fname_ext2_ext1)
    out_fname, ext2 = os.path.splitext(out_fname_ext2)

    if filetype == 'SSMminiSVP':
        fname = out_fname
    elif filetype == 'Idronaut' or filetype == 'CTD90':
        fname = out_fname_ext2
    
    # Find the file in the logbook
    for lnum, fn in enumerate(fn_list):
        if fn == fname:
            if sn_list[lnum] == str(sn_or_type):
                # mod_out_fname = f'{st_list[lnum]}_{sn_list[lnum]}_{fn_list[lnum]}' Old Var
                mod_out_fname = f'{fn_list[lnum]}_{st_list[lnum]}_{sn_list[lnum]}'
                # modify output path
                mod_fname_ext = mod_out_fname + '.svp'
                mod_out_path = os.path.join(out_folder_path, mod_fname_ext)
                
                if mod_out_path is None:
                    raise RuntimeError('Modified path is None. Something wrong with the code or the data')
                return mod_out_path

def convert_SMMcsv_to_PDSsvp(SSMcsv_path, PDSsvp_path):
    # Step 1. Read Sound Speed Manager .csv file
    pdepth = []
    pSV = []
    ptemp = []
    psal = []

    with open(SSMcsv_path, 'r') as f:
        # field order: depth, sound speed, salinity, temperature
        line_start = 5  # previous 5 lines are parts of a header
        filecontent = f.read().splitlines()[line_start:]
        for line in filecontent:
            line_content = line.split(',')
            pdepth.append(float(line_content[0]))
            pSV.append(float(line_content[1]))
            ptemp.append(float(line_content[3]))
            psal.append(float(line_content[2]))

    # Step 2. Create a new PDS2000 .svp and update it by SSM .csv values
    pheader = '[POINTS]\n'
    pnum_end = len(pdepth)
    offset_header = '\n[OFFSET]\n'
    sv_offset_name = 'SVOffset'
    sv_offset = '0'
    sv_offset_line = '{:} = {:}\n\n'.format(sv_offset_name,
                                        sv_offset)

    # field order: depth, sound velocity, temperature, salinity
    with open(PDSsvp_path, 'w') as f:
        f.write(pheader)
        for pnum in range(pnum_end):
            # field order: depth, sound velocity, temperature, salinity
            point = 'Point({:d}) = {:f},{:f},{:f},{:f}\n'.format(pnum+1,
                                                                 pdepth[pnum]*-1, 
                                                                 pSV[pnum], 
                                                                 ptemp[pnum], 
                                                                 psal[pnum])
            f.write(point)
        f.write(offset_header)
        f.write(sv_offset_line)

############################

serial_number_list - list с серийными номерами зондов

data_folder_list1 - list с путями до папок с .csv данными (обработанными в SSM данными SVP зондов)

my_svp_log_path - путь до журнала SVP зондирований

In [9]:
miniSVP_43946 = r'D:\aa_yandexcloud\aa_cloudmbesproceessing\ABP56\CTD&SVP\SoundSpeedManager\43946'
miniSVP_75749 = r'D:\aa_yandexcloud\aa_cloudmbesproceessing\ABP56\CTD&SVP\SoundSpeedManager\75748'

# CTD90_folder = r'\\10.197.125.15\Public\ABP53\abp53_mbes_processing\SVP_data\CTD90\stage3_FZ'
# CTD48_folder = r'\\10.197.125.15\Public\ABP53\abp53_mbes_processing\SVP_data\CTD48\stage3_FZ'

my_svp_log_path = r'D:\aa_yandexcloud\aa_cloudmbesproceessing\ABP56\CTD&SVP\Metadata\ABP56_SVPMeta_ALL.csv'

SSM_csv_list = cli.search(miniSVP_75749, '.csv')

Searching *.csv files in directory:D:\aa_yandexcloud\aa_cloudmbesproceessing\ABP56\CTD&SVP\SoundSpeedManager\75748
Scanned filesets are: 

0 D:\aa_yandexcloud\aa_cloudmbesproceessing\ABP56\CTD&SVP\SoundSpeedManager\75748\V000160.csv
1 D:\aa_yandexcloud\aa_cloudmbesproceessing\ABP56\CTD&SVP\SoundSpeedManager\75748\V000163.csv
2 D:\aa_yandexcloud\aa_cloudmbesproceessing\ABP56\CTD&SVP\SoundSpeedManager\75748\V000165.csv
3 D:\aa_yandexcloud\aa_cloudmbesproceessing\ABP56\CTD&SVP\SoundSpeedManager\75748\V000167.csv
4 D:\aa_yandexcloud\aa_cloudmbesproceessing\ABP56\CTD&SVP\SoundSpeedManager\75748\V000170.csv


In [3]:
SSM_picked = cli.pick_from_list(SSM_csv_list)

Please type two numbers that define the range of filesets you like to copy or move
Please type the first number: 
Please type the second number: 
Do you want to add more elements? (Y/N)


Picked elements:
V000023.csv
V000026.csv
V000028.csv
V000030.csv
V000032.csv
V000034.csv
V000037.csv
V000039.csv
V000042.csv
V000045.csv
V000047.csv
V000049.csv


In [11]:
SSM_picked = SSM_csv_list

# miniSVP_sn = 43946
miniSVP_sn_2 = 75748
# CTD90_type = 'CTD90'
# CTD48_type = 'CTD48'

for SSM_csv_path in SSM_picked:
    PDS_svp_path = modify_output_file_name(miniSVP_sn_2, my_svp_log_path, SSM_csv_path)
    print(PDS_svp_path)
    if PDS_svp_path != None:
        convert_SMMcsv_to_PDSsvp(SSM_csv_path, PDS_svp_path)

D:\aa_yandexcloud\aa_cloudmbesproceessing\ABP56\CTD&SVP\SoundSpeedManager\75748\V000160_ABP56055_75748.svp
D:\aa_yandexcloud\aa_cloudmbesproceessing\ABP56\CTD&SVP\SoundSpeedManager\75748\V000163_ABP56056_75748.svp
D:\aa_yandexcloud\aa_cloudmbesproceessing\ABP56\CTD&SVP\SoundSpeedManager\75748\V000165_ABP56063_75748.svp
D:\aa_yandexcloud\aa_cloudmbesproceessing\ABP56\CTD&SVP\SoundSpeedManager\75748\V000167_ABP56069_75748.svp
D:\aa_yandexcloud\aa_cloudmbesproceessing\ABP56\CTD&SVP\SoundSpeedManager\75748\V000170_ABP56096_75748.svp
